# Setup

## Prerequisites

#### Imports

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt


#### Fix directories, define default variables

In [ ]:
# make sure notebook is ran from src
cwd = os.getcwd()
if not cwd.split('/')[-1] == 'src':
    try:
        os.chdir('src')
    except FileNotFoundError:
        print('Error: please run from src dir or project root')
        sys.exit(1)

# define paths
data_path = '../data/'
csv_path = data_path + 'csv/'
shp_path = data_path + 'shapefiles/'
plot_path = '../plots/'
if not os.path.exists(plot_path):
    os.mkdir(plot_path)

## Data

#### Read in data

##### Trees

In [ ]:
df_trees = pd.read_csv(csv_path + '2015_Street_Tree_Census_-_Tree_Data.csv')

In [ ]:
df_trees.shape

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter('longitude', 'latitude', data=df_trees, s=1e-5, alpha=0.5)
ax.set_axis_off()
ax.set_title('NYC Street Trees (2017)');

##### Traffic

We create the traffic dataframe.

In [ ]:
df_traffic = pd.read_csv(
    csv_path+'Automated_Traffic_Volume_Counts.csv',
    index_col = 'RequestID',
).rename(columns={'WktGeom': 'geometry'})

# only keep data of 2017
df_traffic = df_traffic[df_traffic['Yr'] == 2017]

# create a new column with the average traffic volume for each measurement point
df_traffic['Avg_Vol'] = df_traffic.groupby('geometry')['Vol'].transform('mean')

# drop unnecessary columns and rows
df_traffic = df_traffic.drop(['Yr', 'M', 'D', 'HH', 'MM','Vol'], axis=1)
df_traffic = df_traffic.drop_duplicates(subset='geometry')

# convert geometry column to shapely geometry
df_traffic['geometry'] = df_traffic['geometry'].apply(wkt.loads)

We print how many unique values there are for each column.

In [ ]:
for col in df_traffic.columns:
    print(f'{col:>10}: {len(df_traffic[col].unique())}')

In [ ]:
# cast to geo dataframe
gdf_traffic = gpd.GeoDataFrame(
    df_traffic,
    geometry = 'geometry'
)

# set crs to epsg 2263 NAD83 / New York Long Island (ftUS)
gdf_traffic.crs = 2263

# convert geometry to epsg 4326
gdf_traffic = gdf_traffic.to_crs(epsg=4326)

#### Streets

In [ ]:
gdf_streets = gpd.read_file(
    shp_path + 'NYC Street Centerline (CSCL)/geo_export_f4098162-199c-4100-9c2a-6bc6e35f8734.shp'
).to_crs(epsg=4326)

Visualize the measurement locations on street network.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
gdf_streets.plot(ax=ax, color='grey', alpha=0.5, zorder=1)
gdf_traffic.plot(
    ax = ax,
    column = 'Avg_Vol',
    legend = True,
    legend_kwds = {
        'label': 'Average Traffic Volume',
        'orientation': 'horizontal',
        'shrink': 0.8
    },
    markersize = 5,
    cmap = 'viridis_r',
    zorder = 2
)
ax.set_axis_off()
ax.set_title('Average Traffic Volume in NYC in 2017');

##### Manhattan

In [ ]:
gdf_streets_f = gdf_streets[gdf_streets['borocode'] == '1']
gdf_traffic_f = gdf_traffic[gdf_traffic['Boro'] == 'Manhattan']
df_trees_f = df_trees[df_trees['borough'] == 'Manhattan']

for df, df_f in zip([gdf_streets, gdf_traffic, df_trees], [gdf_streets_f, gdf_traffic_f, df_trees_f]):
    print(f'{df.shape[0]} -> {df_f.shape[0]} ({df_f.shape[0]/df.shape[0]:.2%})')

Visualize again.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
gdf_streets_f.plot(ax=ax, color='grey', alpha=0.5, zorder=1)
gdf_traffic_f.plot(
    ax = ax,
    column = 'Avg_Vol',
    legend = True,
    legend_kwds = {
        'label': 'Average Traffic Volume',
        'orientation': 'horizontal',
        'shrink': 0.5,
    },
    markersize = 5,
    cmap = 'viridis_r',
    zorder = 2
)
ax.set_axis_off()
ax.set_title('Manhattan Traffic Volume in 2017');

In [ ]:
gdf_traffic_f.sort_values('Avg_Vol', ascending=False).head(10)